In [15]:

##Extract
import random
import pyspark
from pyspark.sql import SparkSession, functions
import ConnectionConfig as cc
from pyspark.sql.functions import *

# Set up environment
cc.setupEnvironment()
cc.listEnvironment()
# Start local Spark cluster
spark = cc.startLocalCluster("locks_dim")
spark.getActiveSession() # To get the active Spark session
df_operational_locks_dim = (spark.read
    .format("jdbc")
    .option("driver", cc.get_Property("driver")) 
    .option("url", cc.create_jdbc()) 
    .option("dbtable", "locks")
    .option("user", cc.get_Property("username")) 
    .option("password", cc.get_Property("password")) 
    .load())

df_operational_locks = (df_operational_locks_dim
                        .withColumnRenamed("stationid", "station_id")
                        .withColumnRenamed("lockid", "lock_id").select("station_id", "lock_id"))

# Show the schema and a few records to verify
df_operational_locks.printSchema()
df_operational_locks.show(5)




ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\dobis\AppData\Roaming
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: VIKI
COMSPEC: C:\WINDOWS\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
GOPATH: C:\Users\dobis\go
HOMEDRIVE: C:
HOMEPATH: \Users\dobis
IGCCSVC_DB: AQAAANCMnd8BFdERjHoAwE/Cl+sBAAAAAdGjjLHLGEWRGn9vRsCSowQAAAACAAAAAAAQZgAAAAEAACAAAAD7UhTq8CVvkaUfJ5fXTR5kgkvcIed3OfwPabt1yHYIgAAAAAAOgAAAAAIAACAAAABSFZMhyRZv+fj9Q44MNd0sMMQVbnBwNGmcsxiNFFrAcmAAAAAQjo+0swEYFhn4kypkFiEe0Z+EUeRh+XkMWaxY6J5h885R6WUpGPQjsBjQtDBiDzTJJu/Eu8HKO9rNDQ2HtHCLXjOrbciSueB80zvNehaNnexWcFALkN4Q37FMwos4go9AAAAAsBkth/vA4x8SgTkTjgM6mv3GKidgi5oDWFMyb92y29Ab+MuztrSDRGCJMXxOGeO0p8LJ8WZBYOU66GQTsD0WFw==
IPY_INTERRUPT_EVENT: 1676
JPY_INTERRUPT_EVENT: 1676
JPY_PARENT_PID: 1648
JPY_SESSION_NAME: DWH_LockDim.ipynb
LANG: en_US.UTF-8
LANGUAGE: 
LC_ALL: en_US.UTF-8
LOCALAPPDA

In [16]:
#Transform
df_stations = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver", cc.get_Property("driver")) \
    .option("dbtable", "stations") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .load()

# TRANSFORM: Selecting required columns
df_operational_stations = (df_stations
                        .withColumnRenamed("stationid", "station_id")
                        .withColumnRenamed("stationnr", "station_nr")
                        .withColumnRenamed("gpscoord", "gps_coord")
                        .select("station_id", "station_nr", "street", "number", "zipcode", "district", "gps_coord"))

# Show the result to verify the selected columns
df_operational_stations.show(5)


+----------+----------+--------------------+------+-------+---------+-----------------+
|station_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+----------+--------------------+------+-------+---------+-----------------+
|         1|       026|         Meir (2000)|    84|   2000|ANTWERPEN|(51.2182,4.41241)|
|         2|       019|          ONTBREKEND|    12|   2000|ANTWERPEN| (51.219,4.40405)|
|         3|       020|Groenkerkhofstraa...|     2|   2000|ANTWERPEN|(51.2187,4.40066)|
|         4|       035|Cockerillkaai (2000)|      |   2000|ANTWERPEN|(51.2104,4.38772)|
|         5|       094|        PALEISSTRAAT|   147|   2018|ANTWERPEN|(51.2047,4.39625)|
+----------+----------+--------------------+------+-------+---------+-----------------+
only showing top 5 rows



In [17]:
#Load
#Load
# Save the transformed data to a Delta table
df_joined = df_operational_locks.join(
    df_operational_stations,
    on="station_id",
    how="inner"
)

df_joined.show(5)

+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|station_id|lock_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|        12|    234|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    233|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    232|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    231|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    230|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
only showing top 5 rows



In [19]:
df_joined.write.format("delta").mode("overwrite").saveAsTable("lock_dim")


In [22]:
# Create a temporary view from the Delta table
spark.sql("CREATE OR REPLACE TEMP VIEW lock_dim_view AS SELECT * FROM lock_dim")

lock_dim_view = spark.sql("SELECT * FROM lock_dim_view")
lock_dim_view.show(5)


+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|station_id|lock_id|station_nr|              street|number|zipcode| district|        gps_coord|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
|        12|    217|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    218|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    219|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    220|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
|        12|    221|       120|Schijnpoortweg (2...| 27-29|   2060|ANTWERPEN|(51.2276,4.43923)|
+----------+-------+----------+--------------------+------+-------+---------+-----------------+
only showing top 5 rows

